## Data Ingestion

- Currently using only Word and WordNet Extension columns
- Aggregating & grouping details into json for easy use

In [85]:
import pandas as pd
import json
import numpy as np

In [2]:
df = pd.read_csv('dict4dataset_04-23-22.csv')

In [9]:
df

,Strategy Name,Strategy No.,Category No.,Category Name,Semantic Anchor,Word,WordNet Extension,Phrase Extension,Reprogramming
0,Understanding Common Symptom Indicators,L2d,1.0,Signs of Depression -> Depressed Mood or Dyshp...,Depressed Mood,depress,/,NaN,L2d: No Reprogramming
1,Understanding Common Symptom Indicators,L2d,1.0,Signs of Depression -> Depressed Mood or Dyshp...,Depressed Mood,depressed,"gloomy, grim, blue, dispirited, down, downcast...",NaN,NaN
2,Understanding Common Symptom Indicators,L2d,1.0,Signs of Depression -> Depressed Mood or Dyshp...,Depressed Mood,depression,/,NaN,NaN
3,Understanding Common Symptom Indicators,L2d,1.0,Signs of Depression -> Depressed Mood or Dyshp...,Dyshphoria,NaN,NaN,NaN,NaN
4,Understanding Common Symptom Indicators,L2d,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
82,Understanding Healthy Patterns,L2f,2.0,Coherent narrative words,NaN,therefore,NaN,NaN,NaN
83,Understanding Healthy Patterns,L2f,3.0,Constructive Self-Talk,NaN,help,NaN,NaN,NaN
84,Understanding Healthy Patterns,L2f,3.0,Constructive Self-Talk,NaN,improve,NaN,NaN,NaN
85,Understanding Healthy Patterns,L2f,3.0,Constructive Self-Talk,NaN,work on,NaN,NaN,NaN


In [149]:
df_clean = df.replace({"Word": "/", "WordNet Extension": "/"}, np.nan) \
             .dropna(subset=['Category No.']) \
             .drop(["Phrase Extension", "Reprogramming"], axis=1) \
             .astype({'Category No.': 'int32'}) \
             .sort_values(["Strategy No."])

df_clean

,Strategy Name,Strategy No.,Category No.,Category Name,Semantic Anchor,Word,WordNet Extension
34,Understanding Judgement,L2a,1,Positive Adjectives,NaN,brave,NaN
36,Understanding Judgement,L2a,2,Negative Adjectives,NaN,egoistic,NaN
39,Understanding Cognitive Distortion,L2b,1,Should Statement,should,should,NaN
41,Understanding Cognitive Distortion,L2b,2,All or Nothing Thinking / Overgeneralization,absolutive words,never,NaN
42,Understanding Cognitive Distortion,L2b,2,All or Nothing Thinking / Overgeneralization,absolutive words,always,NaN
...,...,...,...,...,...,...,...
81,Understanding Healthy Patterns,L2f,2,Coherent narrative words,NaN,because,NaN
82,Understanding Healthy Patterns,L2f,2,Coherent narrative words,NaN,therefore,NaN
83,Understanding Healthy Patterns,L2f,3,Constructive Self-Talk,NaN,help,NaN
84,Understanding Healthy Patterns,L2f,3,Constructive Self-Talk,NaN,improve,NaN


In [151]:
df_clean.columns

Index(['Strategy Name', 'Strategy No.', 'Category No.', 'Category Name',
       'Semantic Anchor', 'Word', 'WordNet Extension'],
      dtype='object')

In [176]:
groupped = df_clean.groupby(["Strategy No.", "Category No.", "Category Name", "Semantic Anchor"], dropna=False) \
                   .apply(lambda x: x[["Word","WordNet Extension"]].to_json(orient='columns'))

In [177]:
isinstance(groupped, pd.Series)

True

In [180]:
# groupped.groups
# groupped.index
for index, value in groupped.items():
    print(f"Index : {index}, Value : {json.loads(value)}")

Index : ('L2a', 1, 'Positive Adjectives', nan), Value : {'Word': {'34': 'brave'}, 'WordNet Extension': {'34': None}}
Index : ('L2a', 2, 'Negative Adjectives', nan), Value : {'Word': {'36': 'egoistic'}, 'WordNet Extension': {'36': None}}
Index : ('L2b', 1, 'Should Statement', 'should'), Value : {'Word': {'39': 'should'}, 'WordNet Extension': {'39': None}}
Index : ('L2b', 2, 'All or Nothing Thinking / Overgeneralization', 'absolutive words'), Value : {'Word': {'41': 'never', '42': 'always', '43': 'forever', '45': 'all', '46': 'everyone', '47': 'everything', '48': 'anyone', '49': 'anything', '50': 'only', '51': 'totally', '52': 'no one', '44': 'again', '54': 'nothing', '53': 'none'}, 'WordNet Extension': {'41': None, '42': None, '43': None, '45': None, '46': 'everybody', '47': None, '48': 'anybody', '49': None, '50': None, '51': None, '52': None, '44': None, '54': None, '53': None}}
Index : ('L2b', 3, 'Catastrophizing', nan), Value : {'Word': {'55': 'what if'}, 'WordNet Extension': {'55':

In [103]:
    # TODO: Define target structure:
    
#     dict = [
#       'L2b': {
#          'Category Number' : 1,
#          'Category Name' : 'Should Statement',
#          'Anchors': {
#             'Depressed Mood': {
#                'Words' : ["should"],
#                'WordNet Extension' : [],
#                'Phrase Extension': [],
#                'Reprogramming' : []    
#             },
#             'Dysphoria': {
#                'Words' : [],
#                'WordNet Extension' : [],
#                'Phrase Extension': [],
#                'Reprogramming' : []    
#             }
#          },
#       }, 
    
#       {  'L2c' : {
#          'Cat Number' : 2,
#          'Cat Name' : 'Should Statement',
#          'Words' : ["should"],
#          'WordNet Extension' : [],
#          'Phrase Extension': [],
#          'Reprogramming' : []
#       }
#     ]

In [183]:
agg = []
for index, value in groupped.items():
    out = { 
        "Strategy No." : index[0],
        "Category No." : index[1],
        "Semantic Anchor" : index[2],
        "Words" : json.loads(value)["Word"],
        "WordNet Extension" : json.loads(value)["WordNet Extension"],
    }
    agg.append(out)
print(agg)


[{'Strategy No.': 'L2a', 'Category No.': 1, 'Semantic Anchor': 'Positive Adjectives', 'Words': {'34': 'brave'}, 'WordNet Extension': {'34': None}}, {'Strategy No.': 'L2a', 'Category No.': 2, 'Semantic Anchor': 'Negative Adjectives', 'Words': {'36': 'egoistic'}, 'WordNet Extension': {'36': None}}, {'Strategy No.': 'L2b', 'Category No.': 1, 'Semantic Anchor': 'Should Statement', 'Words': {'39': 'should'}, 'WordNet Extension': {'39': None}}, {'Strategy No.': 'L2b', 'Category No.': 2, 'Semantic Anchor': 'All or Nothing Thinking / Overgeneralization', 'Words': {'41': 'never', '42': 'always', '43': 'forever', '45': 'all', '46': 'everyone', '47': 'everything', '48': 'anyone', '49': 'anything', '50': 'only', '51': 'totally', '52': 'no one', '44': 'again', '54': 'nothing', '53': 'none'}, 'WordNet Extension': {'41': None, '42': None, '43': None, '45': None, '46': 'everybody', '47': None, '48': 'anybody', '49': None, '50': None, '51': None, '52': None, '44': None, '54': None, '53': None}}, {'Stra

## FOR SAVING:

In [195]:
import datetime
ts = datetime.datetime.now().isoformat()

In [196]:
with open(ts+'dict.json', 'w', encoding='utf-8') as f:
    json.dump(agg, f, ensure_ascii=False, indent=4)